In [ ]:
import keras 
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, Activation, BatchNormalization
from keras.optimizers import Adam
import numpy as np
import os
import cv2
import time
from random import shuffle

In [ ]:
inputs = keras.Input(shape = (64,64,3), name="input_layer")

x = Conv2D(32, 3, strides = 2, padding="same")(inputs)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(32, 3, strides = 1, padding="same")(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(64, 3, strides = 2, padding="same")(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(64, 3, strides = 1, padding="same")(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)


x = Conv2D(128, 3, strides = 2, padding="same")(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(128, 3, strides = 1, padding="same")(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(256, 3, strides = 2, padding="same")(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv2D(256, 3, strides = 1, padding="same")(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Flatten()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = Dense(2)(x)
x = Activation("softmax", name="output_layer")(x)

In [ ]:
model = keras.Model(inputs=inputs , outputs=x)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
images_list = []
for root, folder, files in os.walk("/home/grace/Projects/Dog_Vs_Cat/Train_data/"):
    for file in files :
        images_list.append(root + file)
images_list , test = images_list[0:20000] , images_list[20000:]
print("No of Train Images : ", len(images_list))
print("No of Test Images  : ", len(test))

In [ ]:
def train_gen(images_list, batch):
    while True:
        shuffle(images_list)
        total_images = len(images_list)
        for i in range(0, total_images-batch, batch):
            image_batch = []
            label_batch = []
            for image in images_list[i:i+batch]:
                single_image = cv2.imread(image)/255.0 - 0.5
                single_image = cv2.resize(single_image, (64, 64))
                image_batch.append(single_image)
                label = int(image.split("/")[-1].split("_")[0])
                label_batch.append(label) #np.eye(2)[label])
            label_batch = np.array(label_batch, dtype=np.float32)
            image_batch = np.array(image_batch)
            yield image_batch, label_batch

In [ ]:
def test_gen(test, batch) :
    while True:
        test_images = len(test)
        shuffle(test)
        for j in range(0, test_images-batch, batch):
            test_image_batch = []
            test_label_batch = []   
            for test_image in test[j:j+batch]:
                single_test_image = cv2.imread(test_image)/255.0 - 0.5
                single_test_image = cv2.resize(single_test_image, (64, 64))
                test_image_batch.append(single_test_image)
                test_label = int(test_image.split("/")[-1].split("_")[0])
                test_label_batch.append(test_label) #np.eye(2)[test_label])
            test_label_batch = np.array(test_label_batch, dtype=np.float32)
            test_image_batch = np.array(test_image_batch)
            yield test_image_batch, test_label_batch

In [ ]:
batch_size = 32
model.fit_generator(train_gen(images_list, batch=batch_size), 
                    steps_per_epoch = len(images_list) // batch_size,
                    validation_data = test_gen(test, batch=batch_size), 
                    validation_steps = len(test) // batch_size,
                    epochs=25)

In [ ]:
model.save("KerasModel_ILNL.h5")